In [1]:
import numpy as np
import torch
from ferrari.Ferrari import Ferrari
from ferrari.Ferrari_torch import ferrari

In [2]:
def quartic_polynomial(x,a,b,c,d,e):
    p = a*(x**4) + b*(x**3)+ c*(x**2)+  d*x + e
    return p

In [3]:
a = torch.randn(100)*10
b = torch.randn(100)*10
c = torch.randn(100)*10
d = torch.randn(100)*10

x = torch.randn(100)*10
e = -quartic_polynomial(x,a,b,c,d,0)

X = ferrari(a,b,c,d,e)

In [4]:
i = 7
x = Ferrari(a[i],b[i],c[i],d[i],e[i])
print(X[:,i])
print(x)

tensor([-1.7502-3.0531e-08j,  1.1782+3.0531e-08j,  0.4930-1.4345e+00j,
         0.4930+1.4345e+00j])
[-1.7502154 -7.2995171e-08j  1.1781672 +2.0477754e-07j
  0.49299854+1.4344940e+00j  0.49299878-1.4344941e+00j]


In [5]:
err_max = [
    quartic_polynomial(X[0],a,b,c,d,e).real.max(),
    quartic_polynomial(X[1],a,b,c,d,e).real.max(),
    quartic_polynomial(X[2],a,b,c,d,e).real.max(),
    quartic_polynomial(X[3],a,b,c,d,e).real.max(),
]


err_mean = [
    quartic_polynomial(X[0],a,b,c,d,e).real.mean(),
    quartic_polynomial(X[1],a,b,c,d,e).real.mean(),
    quartic_polynomial(X[2],a,b,c,d,e).real.mean(),
    quartic_polynomial(X[3],a,b,c,d,e).real.mean(),
]


In [52]:
err_max

[tensor(28.2706), tensor(0.0215), tensor(0.0215), tensor(2.8349)]

In [53]:
err_mean

[tensor(0.0390), tensor(-0.2402), tensor(-0.2373), tensor(-0.2183)]